In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from cycler import cycler
import mplhep as hep
from src.models import TranAD, iTransformer
from torch.utils.data import Dataset, DataLoader
import torch
from main import convert_to_windows

/Users/lauraboggia/VSCode_projects/TranAD/.conda/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# plt.rcParams['axes.prop_cycle'] = cycler('color', ['#17becf','#8c564b', '#e377c2', '#7f7f7f','#bcbd22', '#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd'])
# plt.style.use([hep.style.ROOT, hep.style.firamath])

In [3]:
iTransf_n_windows = [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, 200, 500, 1000, 2000]
TranAD_n_windows = [10, 20, 30, 40, 50, 60, 70]

In [4]:
N = 25
dataset = 'SMAP'

In [5]:
def load_dataset(dataset):
	loader = []
	for file in ['train', 'test', 'labels']:
		if dataset == 'SMD': file = 'machine-1-1_' + file
		if dataset == 'SMAP': file = 'P-1_' + file
		if dataset == 'MSL': file = 'C-1_' + file
		if dataset == 'UCR': file = '136_' + file
		if dataset == 'NAB': file = 'ec2_request_latency_system_failure_' + file
		loader.append(np.load(os.path.join(f'processed/{dataset}', f'{file}.npy')))
	
	train_loader = DataLoader(loader[0], batch_size=loader[0].shape[0])
	test_loader = DataLoader(loader[1], batch_size=loader[1].shape[0])
	labels = loader[2]
	print('training set shape:', train_loader.dataset.shape)
	print('test set shape:', test_loader.dataset.shape)
	return train_loader, test_loader, labels

In [15]:
from src.plotting import plot_ascore, plot_labels, plot_metrics, compare_labels
from main import backprop
from src.pot import pot_eval

ImportError: cannot import name 'compare_labels' from 'src.plotting' (/Users/lauraboggia/VSCode_projects/TranAD/src/plotting.py)

In [13]:
def replotting(model_name, n_windows, dataset='SMAP', N=25):
    for i, elem in enumerate(n_windows):
        # path = f'studies/{model_name}_{dataset}_2/n_window{elem}/results'
        model_path = f'studies/{model_name}_{dataset}_2/n_window{elem}/checkpoints/model.ckpt'
        plot_path = f'studies/{model_name}_{dataset}_2/n_window{elem}/plots'

        # initialise model etc
        checkpoint = torch.load(model_path)
        if model_name == 'TranAD':
            model = TranAD(N, elem).double()
        elif model_name == 'iTransformer':
            model = iTransformer(N, elem).double()
        else:
            print('pbm with model loading')
            break
        optimizer = torch.optim.AdamW(model.parameters() , lr=model.lr, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 5, 0.9)
        # load model etc
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        scheduler.load_state_dict(checkpoint['scheduler_state_dict'])
        # epoch = checkpoint['epoch']
        # accuracy_list = checkpoint['accuracy_list']
        total_params = sum(p.numel() for p in model.parameters())
        trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
        print(f'total params: {total_params}, trainable params: {trainable_params}')

        train_loader, test_loader, labels = load_dataset(dataset)
        trainD, testD = next(iter(train_loader)), next(iter(test_loader))
        trainO, testO = trainD, testD
        # prepare data
        if model.name in ['Attention', 'DAGMM', 'USAD', 'MSCRED', 'CAE_M', 'GDN', 'MTAD_GAT', 'MAD_GAN', 'iTransformer'] or 'TranAD' in model.name: 
            trainD, testD = convert_to_windows(trainD, model), convert_to_windows(testD, model)
        
        ### Testing phase
        torch.zero_grad = True
        model.eval()
        lossT, _ = backprop(0, model, trainD, trainO, optimizer, scheduler, training=False)  # training loss for POT
        loss, y_pred = backprop(0, model, testD, testO, optimizer, scheduler, training=False)

        preds = []
        for i in range(loss.shape[1]):
            lt, l, ls = lossT[:, i], loss[:, i], labels[:, i]
            result, pred = pot_eval(lt, l, ls, plot_path, f'dim{i}')
            preds.append(pred)
            df_res = pd.DataFrame.from_dict(result, orient='index').T
        lossTfinal, lossFinal = np.mean(lossT, axis=1), np.mean(loss, axis=1)
        labelsFinal = (np.sum(labels, axis=1) >= 1) + 0
        preds = np.array(preds).T
        preds = preds.astype(int)
        df_labels = pd.DataFrame(preds)
        labelspred = (np.sum(preds, axis=1) >= 1) + 0
        print(loss.shape, labelsFinal.shape)
        
        plot_ascore(plot_path, 'ascore_local', loss, labelsFinal)
        plot_labels(plot_path, 'labels_local', labelspred, labelsFinal)
        plot_metrics(plot_path, 'metrics_local', labelspred, labelsFinal)

        result2, pred2 = pot_eval(lossTfinal, lossFinal, labelsFinal, plot_path, f'all_dim')
        labelspred2 = (pred2 >= 1) + 0
        plot_ascore(plot_path, 'ascore_global', lossFinal, labelsFinal)
        plot_labels(plot_path, 'labels_global', labelspred2, labelsFinal)
        plot_metrics(plot_path, 'metrics_global', labelspred2, labelsFinal)
        
        # arr = np.where(labelspred!=labelspred2)
        # print(len(arr[0]), len(np.where(labelspred==labelspred2)[0]))

        compare_labels(plot_path, labels_loc=labelspred, labels_glob=labelspred2, labels=labelsFinal)


In [14]:
replotting('TranAD', TranAD_n_windows, dataset, 25)
replotting('iTransformer', iTransf_n_windows, dataset, 25)

total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
556 7949
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
577 7928
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
535 7970
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
551 7954
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
573 7932
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
599 7906
total params: 57273, trainable params: 57273
training set shape: (2872, 25)
test set shape: (8505, 25)
(8505, 25) (8505,)
602 7903
total params: 47916, trainable params: 47916
training set shape: (2872, 25)
test se